<a href="https://colab.research.google.com/github/serciex/lane-change/blob/main/V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing Prerequisites

In [1]:
!pip install pygame
!pip install highway-env

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 2.6 MB/s eta 0:00:00


Testing

In [2]:
#Import Libraries
from os import truncate
import math
import gymnasium
import highway_env
from matplotlib import pyplot as plt
import pygame
import numpy as np


from IPython.display import Image, display
from collections import deque
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
%matplotlib inline

Environment Definition

In [3]:
# Configure Environment Conditions
config = {
    "lanes_count": 3,
    "lane_width": 3.75,
    "observation": {
        "type": "Kinematics",
        "features": ["x", "y", "vx", "vy", "heading", "lat_off"]
    },
    "action": {"type": "ContinuousAction"},"ego_spawn_random": True,
    "policy_frequency": 10,
}
env = gymnasium.make('highway-v0', render_mode='rgb_array', config=config)
frames = []

# Action Setup
highway_env.envs.common.action.ContinuousAction(env, lateral=True,
                                                longitudinal=True)

In [4]:
# Environment Manager
class ENV(env.__class__):
  """
  s = (v,a,x,y,thetha,id,w,c) ∈ S

  Lateral Agent State:
  Obs Data:
  x = vehicle x position (x)
  y = vehicle y position (y)
  v = vehicle speed (vx)
  thetha = yaw angle (heading)

  Input:
  a = longitudinal acceleration (longitudinal_control)
  id = target lane id
  w = lane width
  c = road curvature

  Extra Data:
  vy = lateral rate (vy)
  delta_lat_deviation = change in lateral deviation (lat_off)

  """
  def __init__(self, obs,a):
    self.obs = obs
    self.a = a

  def ego_state_idm(self):
      ax, ay, avx, avy, athetha, _ = self.obs[0]
      self._ego_state_idm = {"x": ax, "y": ay, "vx": avx,"thetha": athetha,
                         "longitudinal_acceleration":self.a}

      return self._ego_state_idm

  def ego_state_agent(self,target_id):
      ax, ay, avx, avy, athetha, _ = self.obs[0]
      vehicle = env.unwrapped.vehicle

      # Control Parameters
      self.id = target_id

      # Environment Parameters
      self.s,_ = vehicle.lane.local_coordinates(vehicle.position)
      self.w = vehicle.lane.width
      self.c = vehicle.lane.heading_at(np.clip(
          vehicle.lane.local_coordinates(vehicle.position)[0],
          0, vehicle.lane.length))
      self.v = math.sqrt(avx**2+avy**2)

      self._ego_state_agent = {"x": ax, "y": ay, "vx": self.v,"thetha": athetha,
                         "lane_width":self.w,"lane_id":self.id,
                         "self_curvature":self.c,
                         "longitudinal_acceleration":self.a}

      return self._ego_state_agent

  def longitudinal_lead_state(self):
    # Lead State Parameters on the same Lane (5 columns: presence, x, y, vx, vy)
    ego_vehicle = env.unwrapped.vehicle
    lead_vehicle = env.unwrapped.road.neighbour_vehicles(ego_vehicle, lane_index=ego_vehicle.lane_index)[0]
    if lead_vehicle:
      gap = lead_vehicle.position[0] - ego_vehicle.position[0]
      delta_velocity = ego_vehicle.velocity[0] - lead_vehicle.velocity[0]
      self.longitudinal_lead_state = {"x": gap, "vx": delta_velocity}

    else:
      self.longitudinal_lead_state = {"x": 10, "vx": 0}

    return self.longitudinal_lead_state

  #Reward Function
  def reward_function(self, obs_old, obs_new, target_id, w1=1, w2=1, w3=0.05):
    """
    Reward Function:

    Acceleration Reward: r_acce = w1*f_acce(a_yaw)
    a_yaw = lateral acceleration (self.action)

    Rate Reward: r_rate = w2*f_rate(w_yaw)
    w_yaw = lateral rate (vy)

    Time Reward: r_time = w3*f_time (delta_lat_deviation)
    delta_lat_deviation = change in lateral deviation (self.lat_off)

    Reward = Cummulative Sum of r_acce + Cummulative Sum of r_rate + Cummulative Sum of r_time

    """

    self.target_id = ("0","1",target_id)
    target_lane_object = env.unwrapped.road.network.get_lane(self.target_id)
    vehicle_s, _ = env.unwrapped.vehicle.lane.local_coordinates(env.unwrapped.vehicle.position)
    _ , self.delta_lat_deviaton = target_lane_object.local_coordinates(env.unwrapped.vehicle.position)

    obs = obs_new[0]
    obs_old = obs_old[0]

    w_yaw = (obs[0] * obs[3] - obs[1] * obs[2]) / (obs[0]**2 + obs[1]**2 + 1e-8)
    w_yaw_old = (obs_old[0] * obs_old[3] - obs_old[1] * obs_old[2]) / (obs_old[0]**2 + obs_old[1]**2 + 1e-8)

    self.w_acce = (w_yaw-w_yaw_old)*env.unwrapped.config['policy_frequency']

    # Acceleration Reward
    acce_reward = -1*abs(self.w_acce)

    # Rate Reward
    rate_reward = -1*abs(w_yaw)

    # Time Reward
    time_reward = -0.05 * abs(self.delta_lat_deviaton)

    # Overall Reward
    self.reward = w1*acce_reward + w2*rate_reward + w3*time_reward

    return [self.reward, acce_reward, rate_reward, time_reward]

  #Acceleration to Steering angle
  def steering_angle(self, agent_action,L=1):
    """
    Steering Angle: theta = atan(a_yaw/v^2)
    a_yaw = lateral acceleration (agent_action)
    v = vehicle speed (vx)
    """
    self.angle = math.atan(L*agent_action/self.ego_state_idm()['vx']**2)

    return self.angle

Agent Defintion

In [5]:
class Agent(nn.Module):
  """
  s = (x, y, vx, vy, thetha, lane_width, lane_id, self_curvature, longitudinal_acceleration) ∈ S

  Lateral Agent State:
  x = vehicle x position
  y = vehicle y position
  vx = vehicle speed (longitudinal)
  thetha = yaw angle (heading)
  lane_width = width of the lane
  lane_id = target lane id
  self_curvature = road curvature at the vehicle's position
  longitudinal_acceleration = vehicle longitudinal acceleration
  """
  def __init__(self, state_dim):
    super(Agent, self).__init__()
    self.state_dim = state_dim
    # Define Network A & C
    self.networkA = nn.Sequential(
        nn.Linear(self.state_dim, 100),
        nn.Linear(100, 1),
        nn.Softplus()
    )
    self.networkC = nn.Sequential(
        nn.Linear(self.state_dim + 1, 100),
        nn.ReLU(),
        nn.Linear(100, 1),
    )

    # Define Network B
    self.networkB1 = nn.Sequential(
        nn.Linear(self.state_dim, 150),
        nn.ReLU(),
        nn.Linear(150, 1),
    )
    self.networkB2 = nn.Sequential(
        nn.Linear(self.state_dim, 150),
        nn.ReLU(),
        nn.Linear(150, 1),
    )
    self.networkB3 = nn.Sequential(
        nn.Linear(self.state_dim, 150),
        nn.ReLU(),
        nn.Linear(150, 1),
    )

  def forward(self, state, a, terminal):
    """
    Q(s,a) = A(s)*(B(s)-a)^2 + C(s)
    """
    if isinstance(state, dict):
      state_tensor = torch.tensor(
          [state[key] for key in ['vx', 'longitudinal_acceleration', 'x', 'y', 'thetha', 'lane_id', 'lane_width', 'self_curvature']],
          dtype=torch.float32
      ).unsqueeze(0)
    else:
        self.state = state
    # Use a directly if it's already a tensor
    self.a = a if isinstance(a, torch.Tensor) else torch.tensor([[a]], dtype=torch.float32)
    self.terminal_condition = terminal if isinstance(terminal, torch.Tensor) else torch.tensor([[terminal]], dtype=torch.float32)

    # Output of the Networks
    self.A = self.networkA(self.state)
    self.C = self.networkC(torch.concat((self.state, self.terminal_condition), dim=1))
    self.B = torch.max(self.networkB1(self.state) * self.networkB2(self.state),
                         self.networkB3(self.state))

    # Q-function Approximation
    q_value = -self.A * ((self.B - self.a) ** 2) + self.C

    return q_value

  def action(self, state):
      # Convert it to a tensor
      if isinstance(state, dict):
          state_tensor = torch.tensor(
              [state[key] for key in ['vx', 'longitudinal_acceleration', 'x', 'y', 'thetha', 'lane_id', 'lane_width', 'self_curvature']],
              dtype=torch.float32
          ).unsqueeze(0)
      else:
          state_tensor = state
      self.state = state_tensor
      self.B = torch.max(self.networkB1(self.state) * self.networkB2(self.state),
                         self.networkB3(self.state))
      return self.B

Experience Buffer

In [6]:
from collections import deque # Ensure imported

class Experience_Buffer():
    """
    Stores transitions as tuples of NumPy arrays (state_vec, action_vec, reward_vec, next_state_vec, terminal_vec).
    """
    def __init__(self, buffer_size):
        self.buffer_size = buffer_size
        self.buffer = deque(maxlen=self.buffer_size)

    def add(self, state_vec, action, reward, next_state_vec, terminal_condition):
        state_np = np.asarray(state_vec, dtype=np.float32)
        next_state_np = np.asarray(next_state_vec, dtype=np.float32)
        action_np = np.asarray([action], dtype=np.float32)
        reward_np = np.asarray([reward], dtype=np.float32)
        terminal_np = np.asarray([float(terminal_condition)], dtype=np.float32)

        self.transition = (state_np, action_np, reward_np, next_state_np, terminal_np)
        self.buffer.append(self.transition)

    def sample_experience(self, batch_size, device):
        """ Samples batch and returns Tensors. """
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, terminals = zip(*batch)

        states_np = np.stack(states)
        actions_np = np.stack(actions)
        rewards_np = np.stack(rewards)
        next_states_np = np.stack(next_states)
        terminals_np = np.stack(terminals)

        # Convert final batch arrays to Tensors
        states_tensor = torch.tensor(states_np, dtype=torch.float32, device=device)
        actions_tensor = torch.tensor(actions_np, dtype=torch.float32, device=device)
        rewards_tensor = torch.tensor(rewards_np, dtype=torch.float32, device=device)
        next_states_tensor = torch.tensor(next_states_np, dtype=torch.float32, device=device)
        terminals_tensor = torch.tensor(terminals_np, dtype=torch.float32, device=device)

        return states_tensor, actions_tensor, rewards_tensor, next_states_tensor, terminals_tensor

    def size(self):
        return len(self.buffer)

Lateral Controller (Gap Checker)

In [7]:
import numpy as np

class Gap_Controller(env.__class__):
    def __init__(self, obs=None, following_gap_threshold=10.0):
        # Optionally store an initial observation if provided.
        if obs is not None:
            self.obs = obs
        # Threshold for safety gap with the following vehicle
        self.following_gap_threshold = following_gap_threshold

    def lane_checker(self):
        """
        Determines which adjacent lane offers the safest gap for a lane change.
        It examines the current lane, the lane to the left, and the lane to the right,
        returning the lane index that has the largest safe front gap available.
        In addition, if the gap with the following vehicle in the target lane is below the safety
        threshold, the lane change is aborted and the current lane is chosen.

        Returns:
            int: The target lane index (the lane with the largest safe gap, or the current lane)
                 if the candidate lane's following vehicle gap is unsafe.
        """
        lane_number = len(env.unwrapped.road.network.lanes_list())
        ego = env.unwrapped.vehicle

        # Initialize front gap values with a default that indicates an unsafe or non-existent gap.
        gap_current_front = -float('inf')
        gap_right_front = -float('inf')
        gap_left_front = -float('inf')

        # Initialize following gap values (gap between ego and the vehicle behind)
        gap_current_follower = float('inf')
        gap_right_follower = float('inf')
        gap_left_follower = float('inf')

        # Get vehicle neighbours in current lane:
        neighbours_current = env.unwrapped.road.neighbour_vehicles(ego, ego.lane_index)
        # neighbours_current[0] is the vehicle ahead and [1] is the following vehicle.
        if neighbours_current:
            if neighbours_current[0]:
                gap_current_front = neighbours_current[0].position[0] - ego.position[0]
            if len(neighbours_current) > 1 and neighbours_current[1]:
                gap_current_follower = ego.position[0] - neighbours_current[1].position[0]

        # Compute the left and right lane indices.
        current_lane = list(ego.lane_index)
        if current_lane[2] > 0:
            lane_left = (current_lane[0], current_lane[1], current_lane[2] - 1)
        else:
            lane_left = ego.lane_index

        if current_lane[2] < (lane_number - 1):
            lane_right = (current_lane[0], current_lane[1], current_lane[2] + 1)
        else:
            lane_right = ego.lane_index

        # Retrieve neighbour vehicles for the right lane.
        neighbours_right = env.unwrapped.road.neighbour_vehicles(ego, lane_index=lane_right)
        if neighbours_right:
            if neighbours_right[0]:
                gap_right_front = neighbours_right[0].position[0] - ego.position[0]
            if len(neighbours_right) > 1 and neighbours_right[1]:
                gap_right_follower = ego.position[0] - neighbours_right[1].position[0]

        # Retrieve neighbour vehicles for the left lane.
        neighbours_left = env.unwrapped.road.neighbour_vehicles(ego, lane_index=lane_left)
        if neighbours_left:
            if neighbours_left[0]:
                gap_left_front = neighbours_left[0].position[0] - ego.position[0]
            if len(neighbours_left) > 1 and neighbours_left[1]:
                gap_left_follower = ego.position[0] - neighbours_left[1].position[0]

        # Compare the front gaps: current, right, and left.
        front_gaps = [gap_current_front, gap_right_front, gap_left_front]
        best_index = np.argmax(front_gaps)

        # Determine the target lane index based on the best candidate.
        # best_index: 0 => current lane, 1 => right lane, 2 => left lane.
        if best_index == 1:
            candidate_lane = lane_right
            candidate_follower_gap = gap_right_follower
        elif best_index == 2:
            candidate_lane = lane_left
            candidate_follower_gap = gap_left_follower
        else:
            candidate_lane = ego.lane_index
            candidate_follower_gap = gap_current_follower  # in current lane, we don't enforce follower gap condition

        # Check if the candidate lane (if different from the current lane)
        # meets the follower gap condition.
        if candidate_lane != ego.lane_index:
            if candidate_follower_gap < self.following_gap_threshold:
                # The follower gap is too small; do not change lanes.
                target_lane_id = ego.lane_index[2]
            else:
                target_lane_id = candidate_lane[2]
        else:
            target_lane_id = ego.lane_index[2]

        return target_lane_id


Longitudinal Controller

In [8]:
class IDM():
    ''' Intelligent Driving Model for Longitudinal Control

    Control parameters:
      a: maximum acceleration
      b: comfortable deceleration
      delta: acceleration exponent
      T: safe time headway

    Parameters:
      s0: minimum gap
      v0: desired speed

      a (Maximum Acceleration): How fast the vehicle can speed up (m/s²).
      b (Comfortable Deceleration): How smoothly the vehicle slows down (m/s²).
      δ (Acceleration Exponent): The non-linearity factor in acceleration.
      T (Safe Time Headway): The desired minimum following time gap (s).

    Input variables:
      s: current gap
      v: current vehicle speed
      delta_v: relative speed (difference between the vehicle's speed and the leading vehicle's speed)
    '''

    def __init__(self, desired_parameters, control_parameters):
        # Unpack initial parameters: [s0, v0]
        self.s0, self.v0 = desired_parameters
        # Unpack control parameters: [a, b, delta, T]
        self.a, self.b, self.delta, self.T = control_parameters

    def longitudinal_controller(self, input_variables):
        # Unpack input variables: [s, v, delta_v]
        s, v, delta_v = input_variables

        # Small epsilon to account for very small gaps and avoid division by zero
        epsilon = 1e-6

        # Desired gap: s* = s0 + v*T + (v * delta_v) / (2 * sqrt(a * b))
        desired_gap = self.s0 + max(0, v * self.T + ((v * delta_v) / (2 * math.sqrt(self.a * self.b))))

        # IDM acceleration: a_IDM = a * [ 1 - (v / v0)^delta - (s* / s)^2 ]
        acceleration = self.a * (1 - (v / self.v0)**self.delta - (desired_gap / (s + epsilon))**2)

        return acceleration

Initialize Environment

In [10]:
# Initialize Environment
obs, _ = env.reset()

# Initialize Environment Manager and Reward
state_manager = ENV(obs,obs[0][2])

# State Manager for Ego and Lead State
ego_state_idm = state_manager.ego_state_idm()
lead_state = state_manager.longitudinal_lead_state()

# Initial Longitundinal Positions
ego_position_idm = ego_state_idm['x']
lead_position_idm = lead_state['x']

# Initial Velocities (using vx for longitudinal control)
ego_velocity_idm = ego_state_idm['vx']
lead_velocity_idm = lead_state['vx']

Training Loop

Always Active (Option 1)

In [11]:
import imageio
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML
from IPython.display import display
from IPython import display as ipythondisplay
import os

In [19]:
################################################################################
"""Initialization"""
################################################################################
# IDM Initialization
control_parameters = [0.1, 5, 4, 4]  # a, b, δ, T
desired_parameters = [20, 10.0]       # s0, v0

# Set initial Input variables using the gap, current velocity, and relative lead velocity
input_variables = [lead_position_idm, ego_velocity_idm, lead_velocity_idm]

# Setup the IDM Model for Longitudinal control
ego_vehicle_idm = IDM(desired_parameters, control_parameters)

# Agent Initialization
total_steps_taken = 0
lr = 0.01  # Reduced learning rate
gamma = 0.99  # Higher discount factor for better long-term rewards

# Reasonable hyperparameter sizes
buffer_size = int(1e5)  # 100k buffer
batch_size = 256  # Standard batch size
max_timesteps = int(4e4)  # 40k total timesteps

plot_freq_steps = 5000  # Plot every 5000 timesteps

# Epsilon-Greedy Parameters
epsilon = 0.3

# Tracking for performance
timestep_rewards = []  # Store rewards by timestep
timestep_acce_rewards = []  # Store acceleration rewards
timestep_rate_rewards = []  # Store rate rewards
timestep_time_rewards = []  # Store time rewards
global_returns = []  # Cumulative return tracking
loss_history = []  # Loss values
timesteps_list = []  # Track timesteps for plotting
episode_lengths = []  # Track episode lengths
episode_indices = []  # Track episode indices for plotting

target_id = env.unwrapped.vehicle.lane_index[-1]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Environment Information
L = env.unwrapped.vehicle.LENGTH
agent_state = state_manager.ego_state_agent(target_id=target_id)
state_dim = len(agent_state)

# Initialize Buffer
buffer = Experience_Buffer(buffer_size)

# Setup Lateral Control Agent
target_network = Agent(state_dim).to(device)
policy_network = Agent(state_dim).to(device)

# Initialize target network
target_network.load_state_dict(policy_network.state_dict())

# Loss and Criterion Initialization
policy_optimizer = torch.optim.Adam(policy_network.parameters(), lr=lr)
policy_loss_fn = nn.MSELoss()

################################################################################
"""Training & Testing Loop"""
################################################################################
# Track episode count
episode_count = 0

# Track average episode length for reporting
avg_ep_length = 0

# Track episode-specific rewards for calculating statistics
episode_rewards_list = []

# Continue until we reach the maximum number of timesteps
while total_steps_taken < max_timesteps:
    episode_count += 1

    # Reset environment and state manager
    obs, _ = env.reset()
    state_manager = ENV(obs, 0.0)

    # Initial state information
    ego_state_idm = state_manager.ego_state_idm()
    lead_state = state_manager.longitudinal_lead_state()
    agent_state = state_manager.ego_state_agent(target_id=target_id)

    # Update IDM inputs based on initial state
    gap = lead_state['x']
    delta_velocity = lead_state['vx']
    input_variables = [gap, ego_state_idm['vx'], delta_velocity]

    episode_step = 0
    episode_return = 0.0
    done = False

    # Track rewards for this episode specifically
    current_episode_rewards = []

    # Run until episode terminates or we hit max timesteps
    while not done and total_steps_taken < max_timesteps:
        episode_step += 1
        total_steps_taken += 1
        timesteps_list.append(total_steps_taken)

        # Recompute gap
        gap = lead_state['x']

        ########################################################################
        """ Rollout """
        ########################################################################
        gap_control = Gap_Controller(obs, following_gap_threshold=30)
        activated_target_lane = gap_control.lane_checker()

        # Determine target lane
        target_id = activated_target_lane
        agent_state = state_manager.ego_state_agent(target_id=target_id)

        # Epsilon-greedy action selection
        if np.random.random() < epsilon:

            # Exploration: choose random action
            agent_action = np.random.uniform(-1.0, 1.0)

        else:

            # Exploitation: choose best action according to policy
            with torch.no_grad():

                state_tensor = agent_state
                agent_action = policy_network.action(state_tensor).item()

        # Store prior state for buffer
        old_state = agent_state
        obs_old = obs

        # IDM Longitudinal Control
        idm_acceleration = ego_vehicle_idm.longitudinal_controller(input_variables)
        longitudinal_control = idm_acceleration

        # Transform agent action to steering angle
        lateral_control = state_manager.steering_angle(agent_action, L)

        # Combine longitudinal and lateral actions
        action = [longitudinal_control, lateral_control]

        ########################################################################
        """ Storing Data """
        ########################################################################
        obs, reward, done, truncated, info = env.step(action)
        done = done or truncated

        # Update state manager with new observation and applied longitudinal control
        state_manager = ENV(obs, longitudinal_control)

        # Compute reward based on the new state
        reward_components = state_manager.reward_function(obs_old, obs, target_id)
        reward = reward_components[0]

        # Add this reward to the current episode's rewards
        current_episode_rewards.append(reward)

        # Track rewards for each component
        timestep_rewards.append(reward)
        timestep_acce_rewards.append(reward_components[1])  # Acceleration reward
        timestep_rate_rewards.append(reward_components[2])  # Rate reward
        timestep_time_rewards.append(reward_components[3])  # Time reward

        # Update episode return
        episode_return += reward

        ########################################################################
        """ IDM Parameter Update """
        ########################################################################
        ego_state_idm = state_manager.ego_state_idm()
        lead_state = state_manager.longitudinal_lead_state()
        agent_state = state_manager.ego_state_agent(target_id=target_id)

        gap = lead_state['x']
        delta_velocity = lead_state['vx']
        input_variables = [gap, ego_state_idm['vx'], delta_velocity]

        # Convert state dictionaries to vectors before adding to buffer
        old_state_vec = torch.tensor(list(old_state.values()))
        new_state_vec = torch.tensor(list(agent_state.values()))

        # Update experience buffer with vector representations
        terminal_flag = 1.0 if done else 0.0
        buffer.add(old_state_vec, agent_action, reward, new_state_vec, terminal_flag)

        ########################################################################
        """Q-Learning Update"""
        ########################################################################
        if buffer.size() >= batch_size:
            if len(loss_history) == 0:
                print("\n" + "=" * 50)
                print(f"Buffer filled with {buffer.size()} experiences. Starting Q-learning updates...")
                print("=" * 50 + "\n")

            rand_experience = buffer.sample_experience(batch_size=batch_size, device=device)
            states, actions_t, rewards_t, next_states, terminals_t = rand_experience

            # Normalize states and next_states
            states = states
            next_states = next_states

            # Use normalized states for Q-value computation
            current_q_values = policy_network(states, actions_t, terminals_t)

            with torch.no_grad():
                target_network.eval()

                # Get optimal actions for next states
                next_optimal_actions = target_network.action(next_states)

                # Calculate full Q-values using optimal actions
                next_q_values = target_network(
                    next_states,
                    next_optimal_actions,
                    terminals_t
                )

                # Compute target values directly without reshaping
                target_q_values = rewards_t + gamma * next_q_values * (1.0 - terminals_t)

            loss = policy_loss_fn(current_q_values, target_q_values)
            loss_history.append(loss.item())

            policy_optimizer.zero_grad()
            loss.backward()
            policy_optimizer.step()

            if total_steps_taken % 1000 == 0:
                avg_loss = np.mean(loss_history[-100:]) if len(loss_history) >= 100 else np.mean(loss_history)
                print(f"Step: {total_steps_taken:5d}/{max_timesteps} | Recent Avg Loss: {avg_loss:.6f} | Epsilon: {epsilon:.3f}")


        # Plot results based on timesteps
        if total_steps_taken % plot_freq_steps == 0 and total_steps_taken > 0:
            # Create plot directory if it doesn't exist
            plot_dir = "training_plots"
            if not os.path.exists(plot_dir):
                os.makedirs(plot_dir)

            # Plot reward components
            plt.figure(figsize=(12, 8))

            # Plot total rewards
            plt.subplot(2, 2, 1)
            plt.plot(timesteps_list, timestep_rewards)
            plt.xlabel("Timesteps")
            plt.ylabel("Total Reward")
            plt.title("Total Reward per Timestep")

            # Plot acceleration rewards
            plt.subplot(2, 2, 2)
            plt.plot(timesteps_list, timestep_acce_rewards)
            plt.xlabel("Timesteps")
            plt.ylabel("Acceleration Reward")
            plt.title("Acceleration Reward per Timestep")

            # Plot rate rewards
            plt.subplot(2, 2, 3)
            plt.plot(timesteps_list, timestep_rate_rewards)
            plt.xlabel("Timesteps")
            plt.ylabel("Rate Reward")
            plt.title("Rate Reward per Timestep")

            # Plot time rewards
            plt.subplot(2, 2, 4)
            plt.plot(timesteps_list, timestep_time_rewards)
            plt.xlabel("Timesteps")
            plt.ylabel("Time Reward")
            plt.title("Time Reward per Timestep")

            plt.tight_layout()
            plt.savefig(f"{plot_dir}/rewards_timestep_{total_steps_taken}.png")
            plt.show()
            plt.close()

            # Plot loss history if we have losses
            if len(loss_history) > 0:
                plt.figure(figsize=(10, 6))
                plt.plot(range(len(loss_history)), loss_history)
                plt.xlabel("Q-Learning Updates")
                plt.ylabel("Loss")
                plt.title("Loss over Training Updates")
                plt.savefig(f"{plot_dir}/loss_timestep_{total_steps_taken}.png")
                plt.show()
                plt.close()

            # Plot moving average of rewards (with window of 100)
            if len(timestep_rewards) > 100:
                plt.figure(figsize=(10, 6))
                window_size = 100
                moving_avg = np.convolve(timestep_rewards, np.ones(window_size)/window_size, mode='valid')
                plt.plot(range(len(moving_avg)), moving_avg)
                plt.xlabel("Timesteps")
                plt.ylabel(f"Average Reward (Window={window_size})")
                plt.title(f"Moving Average Reward (Window={window_size})")
                plt.savefig(f"{plot_dir}/moving_avg_reward_timestep_{total_steps_taken}.png")
                plt.show()
                plt.close()

            # Plot episode lengths if we have episodes
            if len(episode_lengths) > 0:
                plt.figure(figsize=(10, 6))
                plt.plot(episode_indices, episode_lengths)
                plt.xlabel("Episode")
                plt.ylabel("Length (Steps)")
                plt.title("Episode Lengths")
                plt.savefig(f"{plot_dir}/episode_lengths_{total_steps_taken}.png")
                plt.show()
                plt.close()

            print(f"Plots saved at timestep {total_steps_taken}")

    # End of episode
    global_returns.append(episode_return)
    episode_lengths.append(episode_step)
    episode_indices.append(episode_count)

    # Store this episode's rewards
    episode_rewards_list.append(current_episode_rewards)

    # Calculate average episode length
    avg_ep_length = np.mean(episode_lengths[-100:]) if len(episode_lengths) >= 100 else np.mean(episode_lengths)

    # Get average return from recent episodes
    avg_return = np.mean(global_returns[-100:]) if len(global_returns) >= 100 else np.mean(global_returns)

    # Get most recent loss value
    last_loss = loss_history[-1] if loss_history else "N/A"

    # Calculate average and standard deviation of reward per timestep for this episode
    avg_reward_per_step = np.mean(current_episode_rewards)
    std_reward_per_step = np.std(current_episode_rewards)

    # Print the progress repor
    print(f"Ep {episode_count:3d} | Steps: {episode_step:4d} | Avg Ep Len: {avg_ep_length:.1f} | Total Steps: {total_steps_taken:5d}/{max_timesteps} | "
          f"Ep Return: {episode_return:6.2f} | Avg Return: {avg_return:6.2f} | "
          f"Reward/Step: {avg_reward_per_step:.3f}±{std_reward_per_step:.3f} | "
          f"Epsilon: {epsilon:.3f} | Buffer: {buffer.size():5d} | Loss: {last_loss}")



Buffer filled with 256 experiences. Starting Q-learning updates...

Ep   1 | Steps:  400 | Avg Ep Len: 400.0 | Total Steps:   400/40000 | Ep Return: -397.30 | Avg Return: -397.30 | Reward/Step: -0.993±0.396 | Epsilon: 0.300 | Buffer:   400 | Loss: 0.14047835767269135
Ep   2 | Steps:  400 | Avg Ep Len: 400.0 | Total Steps:   800/40000 | Ep Return: -654.17 | Avg Return: -525.74 | Reward/Step: -1.635±2.010 | Epsilon: 0.300 | Buffer:   800 | Loss: 0.614224910736084
Step:  1000/40000 | Recent Avg Loss: 0.639003 | Epsilon: 0.300
Ep   3 | Steps:  400 | Avg Ep Len: 400.0 | Total Steps:  1200/40000 | Ep Return: -424.44 | Avg Return: -491.97 | Reward/Step: -1.061±0.432 | Epsilon: 0.300 | Buffer:  1200 | Loss: 0.534727931022644


KeyboardInterrupt: 